In [1]:
import os
try:
    os.chdir(os.path.join(os.getcwd(), 'atari'))
    print(os.getcwd())
except:
    pass

In [2]:
#%%
from keras.optimizers import RMSprop
import keras
import numpy as np
from model import get_predefined_model
from ddql import DDQNNGame
from preprocessing import scale_color, wrap_deepmind
import gym
import random
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import pandas as pd
#!pip install gym[atari]

Using TensorFlow backend.


In [3]:
#%%
# %% Set env
WRAPPER = "DM" 
env = gym.make('SpaceInvaders-v0')
path = os.getcwd()

if WRAPPER == "DM":
    env = wrap_deepmind(env, frame_stack=True)
    INPUT_SHAPE = (84, 84, 4)
else:
    from gym.wrappers import AtariPreprocessing
    env = AtariPreprocessing(env)
    INPUT_SHAPE = (84, 84, 1)

In [4]:
#%%
# %% Instantiate model
# ["full", "1,5M", "800k", "300k", "100k"]
MODEL_NAME = "full"
cnn = get_predefined_model(MODEL_NAME, INPUT_SHAPE)
cnn_2 = get_predefined_model(MODEL_NAME, INPUT_SHAPE)

W0924 15:38:11.839224 140557730764608 deprecation_wrapper.py:119] From /home/condauser/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0924 15:38:11.898371 140557730764608 deprecation_wrapper.py:119] From /home/condauser/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0924 15:38:11.902539 140557730764608 deprecation_wrapper.py:119] From /home/condauser/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0924 15:38:12.082099 140557730764608 deprecation_wrapper.py:119] From /home/condauser/anaconda3/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instea

6
added conv
added conv
added conv
added flatten
added dense
added dense
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 20, 20, 32)        8224      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 64)          32832     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               1606144   
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 3078      
Total params: 1,687,206
Trainable params: 1,687,206
Non-trainable par

In [5]:
# %% Setup
path = os.getcwd()

# %% gen path

# GasManija: mover a utils y armar lindo
def gen_path(path, model_name, exp_num):
    return os.path.join(path, "model", MODEL_NAME + "_" + str(exp_num))

exp_num = 1

while os.path.exists(gen_path(path, MODEL_NAME, exp_num)):
    exp_num += 1

saving_path = gen_path(path, MODEL_NAME, exp_num)
os.mkdir(saving_path)

paths = {"model":saving_path + "/model{}.h5".format(MODEL_NAME)}

In [6]:
saving_path

'/home/condauser/work/Gaston/reinforcement-learning-ic/atari/model/full_1'

In [7]:
#%% 
exploration_max = 1.0
exploration_min = 0.1
exploration_steps = 850000 # 800000
exploration_decay = (exploration_max-exploration_min)/exploration_steps

params = {"gamma":0.99, "memory_size": 900000, "batch_size": 32,
            "training_frequency": 4, "target_network_update_frequency": 40000,
            "model_persistence_update_frequency": 10000,
            "replay_start_size": 5000 ,"exploration_test": 0.02,
            "exploration_max": exploration_max, 
            "exploration_min": exploration_min,
            "exploration_steps": exploration_steps, 
            "exploration_decay": exploration_decay}

# Para poder estudiar si se estaba entrenando tuve que cambiar dos parametros
# batch_size lo baje de 32 a 16 - vuelve a 32
# replay_start_size lo baje de 50000 a 500 - lo paso a 5000
# Ademas cambio y agregue muchos mas episodios y corridas

train = True

game_model = DDQNNGame(cnn, cnn_2, env, paths, params, train)

# Agregue los logs a TensorBoard pero no me funciono
#keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True)

env.reset()
frameshistory = []
done = False
model_save_freq = 80 # Ahora son partidas
total_step_limit = 10000000 # 10000000
total_run_limit = 8000
render = False #True
clip = False #True

run = 0
total_step = 0
saves = 0

# TODO Hacer que el modelo cree una carpeta dentro de models para guardar las distintas versiones o que lo
# guarde en el nombre del archivo asi como guardamos las versiones de entrenamiento intermedias 
# TODO Ordenar el codigo
# TODO TOTHINK Hacer esto en un ejecutable .py o una funcion que guarde logs para análisis 
# posterior y tome como argumentos el modelo, cantidad de partidas, cantidad de steps, 
# tiempo de guardado y variables de exploracion 
# TODO TOTHINK Jugar un poco con las variables de exploracion
# TODO Terminar las funciones de evaluacion de modelos

W0924 15:38:12.459656 140557730764608 deprecation_wrapper.py:119] From /home/condauser/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0924 15:38:12.461545 140557730764608 deprecation_wrapper.py:119] From /home/condauser/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



In [ ]:
# %% Main loop
import time
exit = 0
print("Partida número: ", run)
print(game_model._weigths_snapshot())
start = time.time()
performance = []
while exit == 0:
    
    run += 1
    current_state = env.reset()
    if WRAPPER != "DM":
        current_state = np.reshape(current_state, (84, 84, 1))

    step = 0
    score = 0
    while exit == 0:
        if total_step >= total_step_limit:
            print ("Reached total step limit of: " + str(total_step_limit))
            # No sería mejor un break?
            print("Tiempo transcurrido de corrida {}".format(time.time()-start))
            exit = 1
            # agrego a mano
            performance.append({"run":run,
                                "step":step,
                                "score":score})
            pd.DataFrame(performance).to_csv(saving_path+ "/performance.csv", index=False)
        total_step += 1
        step += 1

        if render:
            env.render()

        if run % model_save_freq == 0:
            # Cada model_save_freq de pasos totales guardo los pesos del modelo
            full_saving_path = saving_path + \
                 "/model{}_run{}K_games{}K_copy_{}.h5".format(
                 MODEL_NAME, 
                 total_step_limit/1000000,total_run_limit/1000, 
                 run/10)
            game_model.save_model(full_saving_path)

        action = game_model.move(current_state)
        next_state, reward, terminal, info = env.step(action)
        if WRAPPER != "DM":
            next_state = np.reshape(next_state, (84, 84, 1))

        # next_state = scale_color(next_state)

        if clip:
            reward = np.sign(reward)
        score += reward
        
        game_model.remember(current_state, action, reward, next_state, terminal)
        current_state = next_state

        game_model.step_update(total_step)

        if terminal:
            # game_model.save_run(score, step, run)
            if run % 10 == 0:
                weights_snap = game_model._weigths_snapshot()
                print("Partida número: ", run)
                print("Pesos modelo base: ", weights_snap[0])
                print("Pesos modelo copia: ", weights_snap[1])
                print(score)
                print("Tiempo transcurrido de corrida {}".format(time.time()-start))
            game_model._save_model()
            # Agregado desprolijamente a mano
            performance.append({"run":run,
                                "step":step,
                                "score":score})
            pd.DataFrame(performance).to_csv(saving_path + "/performance.csv", index=False)
            break
        
    # Corto por episodios
    if total_run_limit is not None and run >= total_run_limit:
        print ("Reached total run limit of: " + str(total_run_limit))
        print("Tiempo transcurrido de corrida {}".format(time.time()-start))
        exit = 1
        
final = time.time()

Partida número:  0
([1.3485532, 8.731233, 7.1938925, -85.022285, 2.2475612], [1.3485532, 8.731233, 7.1938925, -85.022285, 2.2475612])
Partida número:  10
Pesos modelo base:  [1.3485532, 8.731233, 7.1938925, -85.022285, 2.2475612]
Pesos modelo copia:  [1.3485532, 8.731233, 7.1938925, -85.022285, 2.2475612]
8.0
Tiempo transcurrido de corrida 16.862382650375366
Partida número:  20
Pesos modelo base:  [-4.969558, -12.628072, -25.078466, -355.64035, 4.4139204]
Pesos modelo copia:  [1.3485532, 8.731233, 7.1938925, -85.022285, 2.2475612]
5.0
Tiempo transcurrido de corrida 67.44700789451599
Partida número:  30
Pesos modelo base:  [-11.740055, -45.632553, -52.64673, -492.47366, 7.432457]
Pesos modelo copia:  [1.3485532, 8.731233, 7.1938925, -85.022285, 2.2475612]
2.0
Tiempo transcurrido de corrida 339.552054643631
Partida número:  40
Pesos modelo base:  [-14.362808, -61.56512, -65.10992, -517.8024, 9.105787]
Pesos modelo copia:  [1.3485532, 8.731233, 7.1938925, -85.022285, 2.2475612]
8.0
Tiempo